## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...
1,2025-07-14-00-54-28 +0000,nypost,Trump quips about signing executive order mand...,https://nypost.com/2025/07/13/us-news/trump-qu...
2,2025-07-14-00-22-17 +0000,startribune,Twins go into All-Star break under .500 with 2...,https://www.startribune.com/minnesota-twins-he...
3,2025-07-13-23-38-18 +0000,startribune,Thousands explore Asian culture while eating t...,https://www.startribune.com/panda-fest-twin-ci...
4,2025-07-13-23-10-41 +0000,nypost,Devoted dad gunned down en route to pick up da...,https://nypost.com/2025/07/13/us-news/nyc-dad-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,7
56,nyc,6
100,mamdani,5
134,president,4
11,he,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...,27
0,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...,26
23,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...,25
38,2025-07-13-12-58-08 +0000,nypost,President Trump reveals what he really wants h...,https://nypost.com/2025/07/13/us-news/presiden...,25
18,2025-07-13-20-11-31 +0000,nypost,Times Square ad for Vivek Ramaswamy rips ‘radi...,https://nypost.com/2025/07/13/us-news/times-sq...,24


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,27,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...
0,26,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...
4,20,2025-07-13-23-10-41 +0000,nypost,Devoted dad gunned down en route to pick up da...,https://nypost.com/2025/07/13/us-news/nyc-dad-...
42,20,2025-07-13-02-44-25 +0000,nypost,Border czar Tom Homan delivers fiery response ...,https://nypost.com/2025/07/12/us-news/border-c...
13,20,2025-07-13-21-33-00 +0000,nypost,Eric Adams rips Andrew Cuomo’s planned bid to ...,https://nypost.com/2025/07/13/us-news/adams-ch...
3,18,2025-07-13-23-38-18 +0000,startribune,Thousands explore Asian culture while eating t...,https://www.startribune.com/panda-fest-twin-ci...
23,18,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...
24,15,2025-07-13-19-05-36 +0000,nypost,Hospice nurse who amputated patient’s foot wit...,https://nypost.com/2025/07/13/us-news/nurse-wh...
7,15,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...
37,14,2025-07-13-13-00-00 +0000,startribune,Tonka Bay begins moving mounds of accidentally...,https://www.startribune.com/tonka-bay-sacred-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
